In [ ]:
#!git clone https://github.com/gamerzahatv/api_rvc_call_flask.git
%cd /content
!git clone -b full_stack https://github.com/gamerzahatv/api_rvc_call_flask.git


In [ ]:
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -P /content/api_rvc_call_flask/Server_AI_backend
%cd /content/api_rvc_call_flask/Server_AI_backend

In [ ]:
!pip install faiss-cpu==1.7.4
!pip install ffmpeg
!pip install fairseq==0.12.2
!pip install praat-parselmouth==0.4.3
!pip install torchcrepe==0.0.21
!pip install pyworld==0.3.2
!pip install flask_ngrok2
!pip install Flask-Cors
!pip install ffmpeg-python
!pip install gdown

In [ ]:
%cd /content/api_rvc_call_flask/Server_AI_backend
import base64 ,io ,csv,zipfile ,gdown ,base64,wave
import  glob, subprocess, torch, os, traceback, sys, warnings, shutil, numpy as np
import json, datetime, requests
import pandas as pd
print(os.path)
sys.path.append('/content/api_rvc_call_flask/Server_AI_backend')
import threading
from time import sleep
from subprocess import Popen
import faiss
from random import shuffle
from config import Config
from upload import upload_model
config = Config()
#from gtts import gTTS
now_dir = os.getcwd()
sys.path.append(now_dir)
tmp = os.path.join(now_dir, "TEMP")
shutil.rmtree(tmp, ignore_errors=True)
shutil.rmtree("%s/runtime/Lib/site-packages/infer_pack" % (now_dir), ignore_errors=True)
shutil.rmtree("%s/runtime/Lib/site-packages/uvr5_pack" % (now_dir), ignore_errors=True)
os.makedirs(tmp, exist_ok=True)
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)
os.environ["TEMP"] = tmp
warnings.filterwarnings("ignore")
torch.manual_seed(114514)
import ffmpeg
ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if (not torch.cuda.is_available()) or ngpu == 0:
    if_gpu_ok = False
else:
    if_gpu_ok = False
    for i in range(ngpu):
        gpu_name = torch.cuda.get_device_name(i)
        if (
            "10" in gpu_name
            or "16" in gpu_name
            or "20" in gpu_name
            or "30" in gpu_name
            or "40" in gpu_name
            or "A2" in gpu_name.upper()
            or "A3" in gpu_name.upper()
            or "A4" in gpu_name.upper()
            or "P4" in gpu_name.upper()
            or "A50" in gpu_name.upper()
            or "A60" in gpu_name.upper()
            or "70" in gpu_name
            or "80" in gpu_name
            or "90" in gpu_name
            or "M4" in gpu_name.upper()
            or "T4" in gpu_name.upper()
            or "TITAN" in gpu_name.upper()
        ):  # A10#A100#V100#A40#P40#M40#K80#A4500
            if_gpu_ok = True  # 至少有一张能用的N卡
            gpu_infos.append("%s\t%s" % (i, gpu_name))
            mem.append(
                int(
                    torch.cuda.get_device_properties(i).total_memory
                    / 1024
                    / 1024
                    / 1024
                    + 0.4
                )
            )
if if_gpu_ok == True and len(gpu_infos) > 0:
    gpu_info = "\n".join(gpu_infos)
    default_batch_size = min(mem) // 2
else:
    gpu_info = ("It's a pity that you don't have a working graphics card to support your training")
    default_batch_size = 1
gpus = "-".join([i[0] for i in gpu_infos])
#from lib.infer_pack.models import (
from infer_pack.models import (
    SynthesizerTrnMs256NSFsid,
    SynthesizerTrnMs256NSFsid_nono,
    SynthesizerTrnMs768NSFsid,
    SynthesizerTrnMs768NSFsid_nono,
)
import soundfile as sf
from fairseq import checkpoint_utils
import logging
from vc_infer_pipeline import VC
from infer_uvr5 import _audio_pre_, _audio_pre_new
from my_utils import load_audio  #Requires the ffmpeg CLI and `ffmpeg-python` package to be installed.

from train.process_ckpt import show_info, change_info, merge, extract_small_model

#config = Config()
# from trainset_preprocess_pipeline import PreProcess
logging.getLogger("numba").setLevel(logging.WARNING)

hubert_model = None
def load_hubert():
    """Load hubert file"""
    global hubert_model
    models, _, _ = checkpoint_utils.load_model_ensemble_and_task(
        ["hubert_base.pt"],
        suffix="",
    )
    hubert_model = models[0]
    hubert_model = hubert_model.to(config.device)
    if config.is_half:
        hubert_model = hubert_model.half()
    else:
        hubert_model = hubert_model.float()
    hubert_model.eval()

#load pth and index
weight_root = "weights"
weight_uvr5_root = "uvr5_weights"
index_root = "logs"
names = []
for name in os.listdir(weight_root):
    if name.endswith(".pth"):
        names.append(name)
index_paths = []
for root, dirs, files in os.walk(index_root, topdown=False):
    for name in files:
        if name.endswith(".index") and "trained" not in name:
            index_paths.append("%s/%s" % (root, name))
uvr5_names = []
for name in os.listdir(weight_uvr5_root):
    if name.endswith(".pth") or "onnx" in name:
        uvr5_names.append(name.replace(".pth", ""))


def vc_single(
    sid,
    input_audio_path,
    f0_up_key,
    f0_file,
    f0_method,
    file_index,
    #file_index2,
    # file_big_npy,
    index_rate,
    filter_radius,
    resample_sr,
    rms_mix_rate,
    protect,
    crepe_hop_length,
):  # spk_item, input_audio0, vc_transform0,f0_file,f0method0
    global tgt_sr, net_g, vc, hubert_model, version
    if input_audio_path is None:
        return "You need to upload an audio", None
    f0_up_key = int(f0_up_key)
    try:
        audio = load_audio(input_audio_path, 16000)
        audio_max = np.abs(audio).max() / 0.95
        if audio_max > 1:
            audio /= audio_max
        times = [0, 0, 0]
        if hubert_model == None:
            load_hubert()
        if_f0 = cpt.get("f0", 1)
        file_index = (
            (
                file_index.strip(" ")
                .strip('"')
                .strip("\n")
                .strip('"')
                .strip(" ")
                .replace("trained", "added")
            )
        )  # 防止小白写错，自动帮他替换掉
        # file_big_npy = (
        #     file_big_npy.strip(" ").strip('"').strip("\n").strip('"').strip(" ")
        # )
        audio_opt = vc.pipeline(
            hubert_model,
            net_g,
            sid,
            audio,
            input_audio_path,
            times,
            f0_up_key,
            f0_method,
            file_index,
            # file_big_npy,
            index_rate,
            if_f0,
            filter_radius,
            tgt_sr,
            resample_sr,
            rms_mix_rate,
            version,
            protect,
            crepe_hop_length,
            f0_file=f0_file,
        )
        if resample_sr >= 16000 and tgt_sr != resample_sr:
            tgt_sr = resample_sr
        index_info = (
            "Using index:%s." % file_index
            if os.path.exists(file_index)
            else "Index not used."
        )
        return "Success.\n %s\nTime:\n npy:%ss, f0:%ss, infer:%ss" % (
            index_info,
            times[0],
            times[1],
            times[2],
        ), (tgt_sr, audio_opt)
    except:
        info = traceback.format_exc()
        print(info)
        return info, (None, None)


def vc_multi(
    sid,
    dir_path,
    opt_root,
    paths,
    f0_up_key,
    f0_method,
    file_index,
    file_index2,
    # file_big_npy,
    index_rate,
    filter_radius,
    resample_sr,
    rms_mix_rate,
    protect,
    format1,
    crepe_hop_length,
):
    try:
        dir_path = (
            dir_path.strip(" ").strip('"').strip("\n").strip('"').strip(" ")
        )  # 防止小白拷路径头尾带了空格和"和回车
        opt_root = opt_root.strip(" ").strip('"').strip("\n").strip('"').strip(" ")
        os.makedirs(opt_root, exist_ok=True)
        try:
            if dir_path != "":
                paths = [os.path.join(dir_path, name) for name in os.listdir(dir_path)]
            else:
                paths = [path.name for path in paths]
        except:
            traceback.print_exc()
            paths = [path.name for path in paths]
        infos = []
        for path in paths:
            info, opt = vc_single(
                sid,
                path,
                f0_up_key,
                None,
                f0_method,
                file_index,
                file_index2,
                # file_big_npy,
                index_rate,
                filter_radius,
                resample_sr,
                rms_mix_rate,
                protect,
                crepe_hop_length
            )
            if "Success" in info:
                try:
                    tgt_sr, audio_opt = opt
                    if format1 in ["wav", "flac"]:
                        sf.write(
                            "%s/%s.%s" % (opt_root, os.path.basename(path), format1),
                            audio_opt,
                            tgt_sr,
                        )
                    else:
                        path = "%s/%s.wav" % (opt_root, os.path.basename(path))
                        sf.write(
                            path,
                            audio_opt,
                            tgt_sr,
                        )
                        if os.path.exists(path):
                            os.system(
                                "ffmpeg -i %s -vn %s -q:a 2 -y"
                                % (path, path[:-4] + ".%s" % format1)
                            )
                except:
                    info += traceback.format_exc()
            infos.append("%s->%s" % (os.path.basename(path), info))
            yield "\n".join(infos)
        yield "\n".join(infos)
    except:
        yield traceback.format_exc()





#get model pth in floder weight  load all
def get_vc(sid):
    global n_spk, tgt_sr, net_g, vc, cpt, version
    if sid == "" or sid == []:
        global hubert_model
        if hubert_model != None:  # 考虑到轮询, 需要加个判断看是否 sid 是由有模型切换到无模型的
            print("clean_empty_cache")
            del net_g, n_spk, vc, hubert_model, tgt_sr  # ,cpt
            hubert_model = net_g = n_spk = vc = hubert_model = tgt_sr = None
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            ###楼下不这么折腾清理不干净
            if_f0 = cpt.get("f0", 1)
            version = cpt.get("version", "v1")
            if version == "v1":
                if if_f0 == 1:
                    net_g = SynthesizerTrnMs256NSFsid(
                        *cpt["config"], is_half=config.is_half
                    )
                else:
                    net_g = SynthesizerTrnMs256NSFsid_nono(*cpt["config"])
            elif version == "v2":
                if if_f0 == 1:
                    net_g = SynthesizerTrnMs768NSFsid(
                        *cpt["config"], is_half=config.is_half
                    )
                else:
                    net_g = SynthesizerTrnMs768NSFsid_nono(*cpt["config"])
            del net_g, cpt
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            cpt = None
        return {"visible": False, "__type__": "update"}
    person = "%s/%s" % (weight_root, sid)
    print("loading %s" % person)   #load model select
    cpt = torch.load(person, map_location="cpu")
    tgt_sr = cpt["config"][-1]
    cpt["config"][-3] = cpt["weight"]["emb_g.weight"].shape[0]  # n_spk
    if_f0 = cpt.get("f0", 1)
    version = cpt.get("version", "v1")
    if version == "v1":
        if if_f0 == 1:
             net_g = SynthesizerTrnMs256NSFsid(*cpt["config"], is_half=config.is_half)
        else:
             net_g = SynthesizerTrnMs256NSFsid_nono(*cpt["config"])
    elif version == "v2":
         if if_f0 == 1:
             net_g = SynthesizerTrnMs768NSFsid(*cpt["config"], is_half=config.is_half)
         else:
             net_g = SynthesizerTrnMs768NSFsid_nono(*cpt["config"])
    del net_g.enc_q
    print(net_g.load_state_dict(cpt["weight"], strict=False))
    print('699999999999999999999999999999999999999999999999999999999999999')
    net_g.eval().to(config.device)
    if config.is_half:
        net_g = net_g.half()
    else:
        net_g = net_g.float()
    vc = VC(tgt_sr, config)
    n_spk = cpt["config"][-3]
    return {"visible": False, "maximum": n_spk, "__type__": "update"}





def clean():
    return {"value": "", "__type__": "update"}






import re as regex
import scipy.io.wavfile as wavfile

#cli_current_page = "HOME"

def cli_split_command(com):
    print("MONGKOL NAMAWRONG")
    exp = r'(?:(?<=\s)|^)"(.*?)"(?=\s|$)|(\S+)'
    split_array = regex.findall(exp, com)
    split_array = [group[0] if group[0] else group[1] for group in split_array]
    return split_array

def execute_generator_function(genObject):
    for _ in genObject: pass

def cli_infer(com):
    # get VC first
    com = cli_split_command(com)
    model_name = com[0]
    source_audio_path = com[1]
    output_file_name = com[2]
    feature_index_path = com[3]
    f0_file = None # Not Implemented Yet

    # Get parameters for inference
    speaker_id = int(com[4])
    transposition = float(com[5])
    f0_method = com[6]
    crepe_hop_length = int(com[7])
    harvest_median_filter = int(com[8])
    resample = int(com[9])
    mix = float(com[10])
    feature_ratio = float(com[11])
    protection_amnt = float(com[12])

    print("Mangio-RVC-Fork Infer-CLI: Starting the inference...")
    vc_data = get_vc(model_name)
    print(vc_data)
    print("Mangio-RVC-Fork Infer-CLI: Performing inference...")
    conversion_data = vc_single(
        speaker_id,
        source_audio_path,
        transposition,
        f0_file,
        f0_method,
        feature_index_path,
        #feature_index_path,
        feature_ratio,
        harvest_median_filter,
        resample,
        mix,
        protection_amnt,
        crepe_hop_length,
    )
    if "Success." in conversion_data[0]:
        print("Mangio-RVC-Fork Infer-CLI: Inference succeeded. Writing to %s/%s..." % ('audio-outputs', output_file_name))
        wavfile.write('%s/%s' % ('audio-outputs', output_file_name), conversion_data[1][0], conversion_data[1][1])
        #print("Mangio-RVC-Fork Infer-CLI: Finished! Saved output to %s/%s" % ('audio-outputs', output_file_name))
        return "Mangio-RVC-Fork Infer-CLI: Finished! Saved output to %s/%s" % ('audio-outputs', output_file_name)
    else:
        print("Mangio-RVC-Fork Infer-CLI: Inference failed. Here's the traceback: ")
        print(conversion_data[0])





def get_presets():
    data = None
    with open('../inference-presets.json', 'r') as file:
        data = json.load(file)
    preset_names = []
    for preset in data['presets']:
        preset_names.append(preset['name'])

    return preset_names

def change_choices2():
    audio_files=[]
    for filename in os.listdir("./audios"):
        if filename.endswith(('.wav','.mp3')):
            audio_files.append(os.path.join('./audios',filename))
    return {"choices": sorted(audio_files), "__type__": "update"}, {"__type__": "update"}

audio_files=[]
for filename in os.listdir("./audios"):
    if filename.endswith(('.wav','.mp3')):
        audio_files.append(os.path.join('./audios',filename))

def get_index():
    if check_for_name() != '':
        chosen_model=sorted(names)[0].split(".")[0]
        logs_path="./logs/"+chosen_model
        if os.path.exists(logs_path):
            for file in os.listdir(logs_path):
                if file.endswith(".index"):
                    return os.path.join(logs_path, file)
            return ''
        else:
            return ''

def get_indexes():
    indexes_list=[]
    for dirpath, dirnames, filenames in os.walk("./logs/"):
        for filename in filenames:
            if filename.endswith(".index"):
                indexes_list.append(os.path.join(dirpath,filename))
    if len(indexes_list) > 0:
        return indexes_list
    else:
        return ''

def get_name():
    if len(audio_files) > 0:
        return sorted(audio_files)[0]
    else:
        return ''



def save_to_wav2(dropbox):
    file_path=dropbox.name
    shutil.move(file_path,'./audios')
    return os.path.join('./audios',os.path.basename(file_path))

def match_index(sid0):
    folder=sid0.split(".")[0]
    parent_dir="./logs/"+folder
    if os.path.exists(parent_dir):
        for filename in os.listdir(parent_dir):
            if filename.endswith(".index"):
                index_path=os.path.join(parent_dir,filename)
                return index_path
    else:
        return ''

def check_for_name():
    if len(names) > 0:
        return sorted(names)[0]
    else:
        return ''

from flask import Flask,jsonify,request
from flask_ngrok2 import run_with_ngrok
import subprocess
from flask_cors import CORS




app = Flask(__name__)
cors = CORS(app)
'''Token ngrok '''
run_with_ngrok(app=app, auth_token="2TzDxTPxT4PsN12y1dTd4RAjz1A_4QQQ9bAUrJ1uS6kmmKnJ4")

##################### MODULE #####################################
'''This is module for chat '''

def process_chat(getinstruction,getinput,getmodel_name):
  try :
    print('chat processing')

    return 'Error'
  except Exception as e:
    return  {"Error": e}




@app.route("/test", methods=["POST"])
def test():
  try:

      content_type = request.headers.get('Content-Type')
      if (content_type == 'application/json'):
        print('yes')
        json = request.json
        print(json)

        if (json):
          result = process_chat(json['instruction'], json['input'],json['model'])
          return {'output': result }
        else:
          return {'output': 'Error' }
  except Exception as e:
      print(e)
      return {
        "Error": e,
      }

@app.route("/nani", methods=["GET"])
def nani():
  try:

    content_type = request.headers.get('Content-Type')
    sample_commands = "serana700_e200_s7200.pth output_audio.wav feature_index_path 1 0.5 f0_method 256 5 44100 0.8 1.0 0.2"
    sample_command = '"serana/serana.pth" "./audios/someguy.mp3" "output_audio.wav" "./logs/serana/serana.index" 0 12 "pm" 120 5  5 0.0  0.0 0.0'

    nani = cli_infer(sample_command)
    return {'output':  nani  }
  except Exception as e:
      print(e)
      return {
        "Error": e,
      }

# send voic image
def prompt_text():
  print('inputhere')





def image_path():
  getimage_dir = os.path.join(os.getcwd(),'voice_model_image','serana','serana.png')
  get_index = os.path.join(os.getcwd(),'logs','serana700_e200_s7200','serana700_v2.index')
  get_weights = os.path.join(os.getcwd(),'weights','serana700_e200_s7200.pth')
  print(getimage_dir)
  print(get_index )
  print(get_weights)
  #print(getimage_dir)
  with open(getimage_dir, "rb") as image_file:
    image_data = image_file.read()
    image_blob = base64.b64encode(image_data).decode('utf-8')
  return image_blob




#upload model
@app.route("/uploadmodel", methods=["POST"])
def uploadmodel():

  try:
    print('loadmodel')
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json'):
      print('yes')
      json_data = request.get_json()
      get_url = json_data.get('url')
      get_modelname =  json_data.get('namemodel')

      if (json):
        try:
          upload_model(get_modelname,get_url)
          return  {
          'output':'uploadmodel Succes',
          }
        except:
          return {
          'output':'uploadmodel error',
          }
  except Exception as e:
    print(e)
    return {
      "Error": e,
    }

#upload audio
@app.route("/uploadaudio", methods=["POST"])
def uploadaudio():

  try:
    print('uploadaudio')
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json'):
      print('yes')
      json_data = request.get_json()
      get_base64 = json_data.get('audiobase64')
      get_audioname = json_data.get('audioname')

      if (json):
        #upload_model(get_modelname,get_url)
        print('down ok')
        print(get_base64)

        path_audio = os.path.join('audios',get_audioname)

        wav_file = open(path_audio, "wb")
        decode_string = base64.b64decode(get_base64)
        wav_file.write(decode_string)

        print("WAV file saved:", path_audio)
        return {
          'output':get_base64,
        }
  except Exception as e:
    print(e)
    return {
      "Error": e,
    }
#image_path()
@app.route("/sendvoicemodel", methods=["POST"])
def send_voice_model():
  try:
      content_type = request.headers.get('Content-Type')
      if (content_type == 'application/json'):
        print('yes')
        json_data = request.get_json()
        print(json_data.get('name'))

        if (json):

          return {
            'voice_model':json_data.get('name'),
            'imageblob': image_path(),
            'imagedesc': 'เป็นตัวละครที่อยู้ในเกม The Elder Scrolls V: Skyrim ใน DLC DAWNGUARD',
            'weight':'weights',
            'index':'index'
          }
        else:
          return {'output': 'Error' }
  except Exception as e:
      print(e)
      return {
        "Error": e,
      }

# refresh audio
@app.route("/refreshaudio", methods=["GET"])
def refreshaudio():
  try:

    list_audio = os.path.join('audios')

    wav_mp3_files = [file for file in os.listdir(list_audio) if file.endswith(('.wav', '.mp3'))]

    # for file in wav_mp3_files:
    #   print(file)

    convert_json = {'output':wav_mp3_files,}
    print(convert_json)
    return  convert_json
  except Exception as e:
    print(e)
    return {
      "Error": e,
    }

def get_datacsv():
  path_db = os.path.join('model_download','DB.csv')
  df = pd.read_csv(path_db)
  print(df.to_string())
  # Assuming 'model' is the column name and 'serana' is the row name
  # You can use the loc function to access the specific value
  value = df.loc[df['columnname'] == 'serana', 'modelname'].values[0]
  # 'value' will now contain the value from the 'model' column where 'serana' is in the specified column
  print(value)

# refresh model and audio  get in csv file
@app.route("/refrehconvert", methods=["GET"])
def refreshconvert():
  try:
    if os.path.exists(os.path.join('model_download','DB.csv')):
      print('HAVE MODEL CSV')
      get_datacsv()
    else:
      with open(os.path.join('model_download','DB.csv'), 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['modelname', 'pth_path', 'index_path'])  # Writing headers if the file is created
        print('Create DB MODEL SUCESS')
        get_datacsv()
  except Exception as e:
    print(e)
    return {
      "Error": e,
    }
if __name__ == '__main__':
    app.run()


In [ ]:
!python -v


KeyboardInterrupt
>>> ^C
